In [66]:
import nest
import csv
import pickle
import numpy as np
import pandas as pd
import os
from collections import defaultdict
import matplotlib.pyplot as plt
nest.ResetKernel()

In [67]:
class EINetwork:
    def __init__(self) -> None:
        self.unique_id = []
        self.neurons = {}
        self.connections = defaultdict(list)
        self.num_neurons = 0
        
        # count of inh and exh synapse
        self.nI = 0
        self.nE = 0
        
        self.neuron_model = 'iaf_psc_alpha'
        
        nest.CopyModel('stdp_synapse_hom', 'e_syn')
        nest.CopyModel('static_synapse', 'i_syn')
    
    def process_and_cache_data_csv(
        self,
        csv_file_path='../connectome_materials/connections_no_threshold.csv',
        cache_file_path='../cache_file/csv_read_to_cache.pkl'
    ):
        if os.path.exists(cache_file_path):
            print('\n*** Loading connections data from cache... ***')
            with open(cache_file_path, 'rb') as f:
                self.unique_id, self.connections = pickle.load(f)
                return self.unique_id, self.connections
            
        print('\n*** Processing CSV file... ***')
        
        data = pd.read_csv(csv_file_path)
        data = data.drop_duplicates(subset=['pre_root_id', 'post_root_id'], keep='last')
        self.unique_id = list(set(data['pre_root_id']).union(set(data['post_root_id'])))
        print('\n*** Unique_id Completed ***')
        
        def convert_nt_type(nt_type):
            return 'i_syn' if nt_type == 'GABA' else 'e_syn'
            
        grouped = data.groupby('pre_root_id')
        for pre_id, group in grouped:
            self.connections[pre_id] = group[['post_root_id', 'syn_count', 'nt_type']].values.tolist()
            self.connections[pre_id] = [
                [post_id, syn_count, convert_nt_type(nt_type)]
                for post_id, syn_count, nt_type in self.connections[pre_id]
            ]
        
        print('\n*** Saving processed data to cache... ***')
        with open(cache_file_path, 'wb') as f:
            pickle.dump((self.unique_id, self.connections), f)
        
        return self.unique_id, self.connections
            
    
    def Create_and_Connect(self):
        print('\n*** Creating Neurons... ***')
        all_neurons = nest.Create(self.neuron_model, n=len(self.unique_id))
        
        print(f'\n*** {len(self.unique_id)} neurons have been created ***')
        
        print(f'\n*** Building neurons dictionary... ***')
        self.neurons = dict(zip(self.unique_id, all_neurons))
        
        print(f'\n*** Dictionary has been created ***')
        
        print(f'\n*** Creating Connections ***')
        
        for pre_id, group in self.connections.items():
            
            pre_neuron = self.neurons[pre_id]
            group_array = np.array(group)
            
            post_ids = group_array[:, 0].astype(int)
            syn_counts = group_array[:, 1].astype(int)
            nt_types = group_array[:, 2]
            post_neuron = np.array([self.neurons[post_id] for post_id in post_ids])
            post_neuron = post_neuron.flatten()
            
            return post_neuron, all_neurons
            break

        
        
        
        

In [68]:
MyNETwork = EINetwork()
unique_id, connections = MyNETwork.process_and_cache_data_csv()
post_neuron, all_neurons = MyNETwork.Create_and_Connect()




*** Loading connections data from cache... ***

*** Creating Neurons... ***

*** 138639 neurons have been created ***

*** Building neurons dictionary... ***

*** Dictionary has been created ***

*** Creating Connections ***


In [73]:
print(all_neurons[post_neuron])

NESTErrors.BadProperty: BadProperty in SLI function take_array_index